In [8]:
!COPY "..\TransferLearning\data_setup.py" .
!COPY "..\TransferLearning\engine.py" .

        1 file(s) copied.
        1 file(s) copied.


In [1]:
import torch
import torch.nn as nn

import torchvision
from torchvision import transforms

from torchinfo import summary
import matplotlib.pyplot as plt

import data_setup, engine

In [2]:
def set_seed(seed: int = 82):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
set_seed()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from pathlib import Path
import zipfile
import requests

def download_zip_dataset(data_dir: str, 
                                              source: str,
                                              destination: str,
                                              remove_zipfile: bool = True) -> Path:
    data_path = Path(data_dir)
    image_path = data_path / destination
    
    if image_path.is_dir():
        print(f"[INFO] {destination} path has existed. Needn't download more!")
        
    else:
        print(f"[INFO] {destination} path is not found. Creating one...")
        image_path.mkdir(exist_ok=True, parents=True)
        
        with open(image_path + ".zip", "wb") as f:
                print(f"Downloading {destination} zipfile")
                req = requests.get(source)
                f.write(req.content)
                
        with zipfile.ZipFile(image_path + ".zip", "r") as zip_ref:
            print(f"Extracting zip file {destination}...")
            zip_ref.extractall(image_path)
            
        if remove_zipfile:
            print(f"Remove zip file")
            os.remove(image_path + ".zip")
        else:
            print(f"Remain zip file")

data_dir = Path("../data")
image_dir = data_dir / "pizza_steak_sushi"
SOURCE = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip"
download_zip_dataset(data_dir, source=SOURCE, destination="pizza_steak_sushi")

[INFO] pizza_steak_sushi path has existed. Needn't download more!


In [4]:
train_dir = image_dir / "train"
test_dir = image_dir / "test"

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                             std=[0.229, 0.224, 0.225])

manual_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize
])

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=manual_transforms,
    batch_size=16
)

train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x1c6345c6f40>,
 ['pizza', 'steak', 'sushi'])

In [5]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
models = torchvision.models.efficientnet_b0(weights=weights)
models

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [6]:
for param in models.features.parameters():
    param.requires_grad = False
    
def summary_models(models) :
    print(summary(model=models, 
                  input_size=(8, 3, 224, 224),
                  col_names=["input_size", "output_size", "trainable", "num_params"],
                  row_settings=["var_names"]))

In [7]:
num_classifier = len(class_names)

models.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features = 1280, 
                      out_features = num_classifier)).to(device)

In [8]:
summary_models(models)

Layer (type (var_name))                                      Input Shape               Output Shape              Trainable                 Param #
EfficientNet (EfficientNet)                                  [8, 3, 224, 224]          [8, 3]                    Partial                   --
├─Sequential (features)                                      [8, 3, 224, 224]          [8, 1280, 7, 7]           False                     --
│    └─Conv2dNormActivation (0)                              [8, 3, 224, 224]          [8, 32, 112, 112]         False                     --
│    │    └─Conv2d (0)                                       [8, 3, 224, 224]          [8, 32, 112, 112]         False                     (864)
│    │    └─BatchNorm2d (1)                                  [8, 32, 112, 112]         [8, 32, 112, 112]         False                     (64)
│    │    └─SiLU (2)                                         [8, 32, 112, 112]         [8, 32, 112, 112]         --                       

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(models.parameters(), lr=1e-4)

In [34]:
!pip install tensorboard

In [11]:
from torch.utils.tensorboard import SummaryWriter

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

SystemError: initialization of _pywrap_checkpoint_reader raised unreported exception